In [2]:
import pandas as pd

df3 = pd.read_table('blast_uniprot_results.txt', header=None)
# the default outfmt 6 columns
default_outfmt6_cols = 'qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore'.strip().split(' ')
df3.columns = default_outfmt6_cols

In [3]:
len(df3)

1484868

In [4]:
# filter for evalue <= 0.001
df_filtered = df3[df3['evalue'] <= 0.05]

# sort by bitscore
df_filtered.sort_values(by='bitscore', ascending=False, inplace=True)

/Users/stronglab2/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [ ]:
len(df_filtered)

In [ ]:
df_filtered.head()

In [5]:
hits = df_filtered['qseqid']
uniq_hits = list(set(hits))
len(uniq_hits)

16694

### Load in Uniprot Protein Sequence IDs

In [6]:
from Bio import SeqIO
ids = []
names = []
for seq_record in SeqIO.parse("viral.1.protein.faa", "fasta"):
    ids.append(seq_record.id)
    names.append(seq_record.description)
    
viral_names = [x.split('[', 1)[1].split(']')[0] for x in names]

In [7]:
df = pd.DataFrame({'IDs': ids, 
                   'Viral Names': viral_names, 
                   'CompleteRecord': names}, columns=['IDs','Viral Names', 'Complete Record'])

In [8]:
y = df['Viral Names'].str.contains('phage')
df2=df[y]
len(df2)

155606

In [9]:
df2['Virulence'] = [1 if x in uniq_hits else 0 for x in df2['IDs']]

/Users/stronglab2/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [10]:
## Group by
test = df2.groupby('Viral Names').sum()

In [11]:
## Group by Bacterial Genus
genus_name = test.index
demo2 = [x.split(' ',1)[0] for x in genus_name]

In [12]:
test['Genus'] = demo2

In [13]:
Virulence_Avg = test.groupby('Genus').mean()
Virulence_Avg = Virulence_Avg.sort_values(by=['Virulence'], ascending=False)
Virulence_Avg.columns = ['Avg Number of Virulent Matches']
Virulence_Avg.head()

,Avg Number of Virulent Matches
Genus,
Aureococcus,44.000000
Prochlorococcus,28.000000
Synechococcus,18.357143
Campylobacter,18.000000
Cronobacter,17.200000


In [15]:
Number_of_phages = test.groupby('Genus').count()
Number_of_phages = Number_of_phages.sort_values(by=['Virulence'], ascending=False)
Number_of_phages.columns = ['Number of Phages in Genus']
Number_of_phages.head()

,Number of Phages in Genus
Genus,
Mycobacterium,249
Pseudomonas,112
Escherichia,92
Bacillus,82
Salmonella,67


In [20]:
### Number of Viruses over 30 in Genus
x = Number_of_phages[Number_of_phages['Number of Phages in Genus']>=30]
abundant_phages = x.index

In [21]:
Abundant_df = Virulence_Avg[(Virulence_Avg.index).isin(abundant_phages)]
Abundant_df

,Avg Number of Virulent Matches
Genus,
Bacillus,16.304878
Escherichia,11.054348
Staphylococcus,8.724138
Salmonella,8.417910
Mycobacterium,8.048193
Vibrio,7.818182
Enterobacteria,7.050847
Gordonia,5.938462
Lactobacillus,5.848485
